## 過去のNotebook

[日本語][EDA] JP Tokyo Stock データ概要を確認 その1 https://www.kaggle.com/code/tatsuyafujii/eda-jp-tokyo-stock-1

In [ ]:
!pip install japanize_matplotlib

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from lightgbm import LGBMRegressor
import optuna.integration.lightgbm as lgb
import seaborn as sns
import japanize_matplotlib
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

## メインデータ

In [ ]:
prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv", parse_dates=["Date"])
print(prices.shape)
prices.head()

## 概観

In [ ]:
## 翌日、翌々日の終値を取得（Target候補）
df = prices.copy()
df["Close_adj"] = df.groupby("SecuritiesCode").apply(lambda d:d["Close"]/d["AdjustmentFactor"].cumprod().shift().fillna(1)).reset_index("SecuritiesCode",drop=True)
df["翌日終値"] = df.groupby("SecuritiesCode")["Close_adj"].shift(-1)
df["翌々日終値"] = df.groupby("SecuritiesCode")["Close_adj"].shift(-2)
df["R"] = (df["翌々日終値"] - df["翌日終値"]) / df["翌日終値"]
(df["R"]-df["Target"]).describe()

In [ ]:
## ある一つの銘柄に注目して確認
d = df[df["SecuritiesCode"]==4923].copy()

sns.lineplot(x=d.Date, y=d.R, label="収益率")
sns.lineplot(x=d.Date, y=d.Target, label="Target")
plt.show()

# 移動平均線と出来高
一般的な移動平均線 5,25,75日でみてみる。ただ、データ数的に75日はほぼ意味がない


### 考察
・デッドクロス5日が25日線を突き抜けたところでさらに下落拡大。<br>
・出来高が大きく増大し、株主が大きく入れ替わり、そこを底として上昇に転じている。<br>
 逆に、出来高をともなって、下落（ストップ安？）した際には再度デッドクロスが発生。下落基調に転換。（再度出来高が増えた4月末まで）<br>
・ただしデッドクロスやゴールデンクロスの翌日、翌々日は上がるもしくは下がるとは限らないのと、クロスのデータ点数が少なすぎて、単体では使えない。<br>


In [ ]:
fig,ax1=plt.subplots(figsize=(16,4))
ax2=plt.twinx(ax1)

def MA(series, window):
    return series.rolling(window).mean()
# 描画する
ax1.plot(d.Date, d.Close_adj, label="株価")
ax1.plot(d.Date, d.Close_adj.rolling(5).mean(), label="5日移動平均線")
ax1.plot(d.Date, d.Close_adj.rolling(25).mean(), label="25日移動平均線")
ax1.plot(d.Date, d.Close_adj.rolling(75).mean(), label="75日移動平均線")
plt.legend()
s = (d.Close_adj.rolling(5).mean() - d.Close_adj.rolling(25).mean())
s = s/s.abs()
d["移動平均線クロス"] = (s - s.shift(1))
ax1.plot(d.Date,d.Close_adj.mean()+ d["移動平均線クロス"]*d.Close_adj.std(), label="ゴールデン・デッドクロス")

ax2.bar(d.Date, d.Volume, label="出来高", color="blue")

# グラフ周りの設定
plt.title('株価推移')
ax1.set_xlabel('年月')
ax1.set_ylabel('株価')
ax2.set_ylabel('出来高')

plt.grid()
handler1, label1 = ax1.get_legend_handles_labels()
handler2, label2 = ax2.get_legend_handles_labels()
ax1.legend(handler1 + handler2, label1 + label2)
plt.show()

display(d.groupby("移動平均線クロス")["Target"].describe())

### シミュレーション
単純にデッドクロス銘柄を売り、ゴールデンクロス銘柄を買い、としてみる<br>
※ 期間が短いため、5日線、25日線を基準とする。<br>
   また、実際にはゴールデンクロスの翌日、翌々日に売り買いするため、反動で逆効果の可能性もありうる。<br>
   そのため、ゴールデンクロス、デッドクロスから0－4日程度ずらした場合も確認。<br>

In [ ]:
def simulate(d, lag=0):
    series = d.Close_adj
    target = d.Target
    assert (series.index == target.index).all()
    s = MA(series, 5) - MA(series, 25)
    s = s/s.abs()
    cross = (s - s.shift(1)).rename("Cross")
    data =  pd.concat([cross.shift(lag), target], axis=1).groupby("Cross")[target.name].sum()
    return data[data.index>0].sum() - data[data.index<0].sum()

result = {}
for i in range(5):
    result[i] = df.groupby("SecuritiesCode").apply(lambda _d: simulate(_d, i))

In [ ]:
import scipy.stats as stats

for i in range(5):
    print(i)
    print(result[i].sum())
    result[i].hist(bins=np.arange(50)/100 - 0.25)
    plt.show()
    
    fig = plt.subplots(figsize=(8,8)) 
    stats.probplot(result[i], dist="norm", plot=plt)
    plt.show()
    print("S-W検定",stats.shapiro(result[i]))
    print("K-S検定",stats.kstest(result[i], "norm"))


### 検証
統計的に有意な差かどうか検証。上記から正規分布とは仮定できそうにない。<br>
ノンパラメトリックに勝ち負けで考えてみる。<br>
2000銘柄で、正（勝ち）、負（負け）が偶然起こる範囲かどうか。<br>
ウィルコクソンの符号付き順位和検定<br>
n=1,2の時は良さそう？　有意水準を 0.01/5 としても有意。<br>

In [ ]:
for i in range(5):
    print(i, result[i].mean())
    print(stats.wilcoxon(result[i], y=None, zero_method='wilcox', correction=False))

## 結論
1日、あるいは2日前にゴールデンクロス、デッドクロスした銘柄を買いあるいは売ると良い。<br>

# 移動平均線乖離率
一般に、移動平均乖離率が大きくなると戻しになる可能性が高い。ただしどこから乖離率大と判断するかは銘柄による<br>
ディフェンシブ銘柄だと10％に年に1度達する程度だが、髙いボラの銘柄なら20％オーバーも頻繁に起こる。<br>

### 考察
日足25日を例として<br>
・この銘柄は比較的ボラティリティが小さい。4ヶ月間で乖離率プラスマイナス3％に達するかどうか<br>
・0％付近をずっと動いている様に見えるので、大きく乖離すると反動で戻る可能性が高い。<br>
・ただし、翌日／翌々日に明確な反応はしていない。散布図見ても、乖離率とリターンに明確な傾向はない<br>

In [ ]:
def MA(series, window):
    return series.rolling(window).mean()

def DMA(series, window=25):
    return series/MA(series, window) - 1

fig,ax1=plt.subplots(figsize=(16,4))
ax2=plt.twinx(ax1)

# 描画する
ax1.plot(d.Date, d.Close_adj, label="株価", color="blue")
ax1.plot(d.Date, MA(d.Close_adj, 25), label="25日移動平均線", color="orange")

d["移動平均線乖離率"] = DMA(d.Close_adj, 25)
ax2.plot(d.Date, d["移動平均線乖離率"], label="25日移動平均線乖離率", color="red")

# グラフ周りの設定
plt.title('株価推移')
ax1.set_xlabel('年月')
ax1.set_ylabel('株価')
ax2.set_ylabel('25日移動平均線乖離率')

# 表示範囲の設定
#ax2.set_ylim(-0.05,0.051)
# gridの設定
plt.grid()

handler1, label1 = ax1.get_legend_handles_labels()
handler2, label2 = ax2.get_legend_handles_labels()
ax1.legend(handler1 + handler2, label1 + label2)
plt.show()

print("25日移動平均線乖離率　統計量")
display(d["移動平均線乖離率"].describe())
(d["移動平均線乖離率"]).plot.box()
plt.show()

sns.scatterplot(x="移動平均線乖離率", y="Target", data=d)

### シミュレーション
単純に移動平均線が一定以上乖離したところで逆張りしてみる<br>
※ 期間が短いため、25日線を基準とする。<br>
　　どこを切り返しの基準とするかは銘柄次第だが、一旦5%～25％で比較してみる。<br>
   最初の2ヶ月でボラをみてしきい値設定する方が良いかもしれない。<br>

In [ ]:
def simulate(d, threshold=0.1):
    series = d.Close_adj
    target = d.Target
    s = DMA(series, 25)
    flg = (s.abs()>=threshold)
    s = (s/s.abs() * flg).rename("DMA")
    data =  pd.concat([s, target], axis=1).groupby("DMA")[target.name].sum()
    return - data[data.index>0].sum() + data[data.index<0].sum()
#simulate(d)
result = {}
for i in range(5):
    result[i] = df.groupby("SecuritiesCode").apply(lambda _d: simulate(_d, 0.05*(i + 1)))

In [ ]:
import scipy.stats as stats

for i in range(5):
    print(i)
    print("合計",result[i].sum())
    print("平均",result[i][result[i]!=0].mean())
    result[i][result[i]!=0].hist(bins=np.arange(50)/100 - 0.25)
    plt.show()
    
    fig = plt.subplots(figsize=(8,8)) 
    stats.probplot(result[i][result[i]!=0], dist="norm", plot=plt)
    plt.show()
    print("S-W検定",stats.shapiro(result[i]))
    print("K-S検定",stats.kstest(result[i], "norm"))


### 検証
統計的に有意な差かどうか検証。上記から正規分布とは仮定できそうにない。<br>
ノンパラメトリックに勝ち負けで考えてみる。<br>
2000銘柄で、正（勝ち）、負（負け）が偶然起こる範囲かどうか。<br>
ウィルコクソンの符号付き順位和検定<br>
5%をしきい値としたときでも十分勝てている。　有意水準を 0.01/5 としても有意。<br>

In [ ]:
for i in range(5):
    print(i, result[i][result[i]!=0].mean())
    print(stats.wilcoxon(result[i], y=None, zero_method='wilcox', correction=False))

## 結論
（銘柄によって本来は変えるべきだろうが）5％ぐらい乖離したら逆張りで良い。<br>
10％など乖離すればするほど勝率が上がるかというとそうでもない。むしろそれだけ乖離するからには乖離するだけの理由があるということか？<br>
トレンド状態かボックス状態での乖離かを判定した上で用いたほうが勝率は高そう。